In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import clear_output

# Function to simulate and visualize infection spread with isolation, vaccination, and basic infection parameters
def simulate_infection_graph(population_size, initial_infected, R0, recovery_rate, isolation_rate, vaccination_rate, vaccine_efficacy, days):
    """
    Parameters:
    - population_size: Total number of nodes (individuals) in the graph. Represents the overall population.
    - initial_infected: Number of initially infected individuals to start the simulation.
    - R0: Basic reproduction number, representing the average number of people an infected person will infect in a fully susceptible population.
    - recovery_rate: Probability of an infected individual recovering each day.
    - isolation_rate: Fraction of the population that isolates, reducing their chance of spreading infection.
    - vaccination_rate: Proportion of the population vaccinated, reducing susceptibility to infection.
    - vaccine_efficacy: Effectiveness of the vaccine, reducing the infection probability for vaccinated individuals.
    - days: Number of days to run the simulation.

    This function simulates infection spread on a graph network, visualizing the status of nodes (susceptible, infected, recovered).
    """

    # Create a random graph to model connections between individuals
    G = nx.erdos_renyi_graph(population_size, 0.05)  # Random graph with probability of edge = 0.05

    # Initialize node attributes: susceptibility, isolation, and vaccination status
    for node in G.nodes:
        G.nodes[node]['state'] = 'susceptible'  # Default state
        # Determine if the node is isolated based on isolation_rate
        G.nodes[node]['isolated'] = np.random.rand() < isolation_rate
        # Determine if the node is vaccinated based on vaccination_rate
        G.nodes[node]['vaccinated'] = np.random.rand() < vaccination_rate

    # Infect the initial infected nodes
    infected_nodes = np.random.choice(list(G.nodes), initial_infected, replace=False)
    for node in infected_nodes:
        G.nodes[node]['state'] = 'infected'

    # Prepare the graph layout to maintain consistent node positioning across plots
    pos = nx.spring_layout(G)

    # Run the infection spread simulation for the specified number of days
    for day in range(days):
        new_infected = []
        # Iterate over infected nodes to try infecting their neighbors
        for node in G.nodes:
            if G.nodes[node]['state'] == 'infected':
                # Attempt infection for each susceptible neighbor
                for neighbor in G.neighbors(node):
                    if G.nodes[neighbor]['state'] == 'susceptible':
                        # Set the baseline infection probability
                        infection_probability = R0 / 10  # Scale R0 down for a daily spread model

                        # Adjust probability for isolation and vaccination
                        if G.nodes[node]['isolated']:
                            infection_probability *= 0.5  # Reduce spread potential if isolated
                        if G.nodes[neighbor]['vaccinated']:
                            infection_probability *= (1 - vaccine_efficacy)  # Reduce spread potential if vaccinated

                        # Infect neighbor based on the calculated probability
                        if np.random.rand() < infection_probability:
                            new_infected.append(neighbor)

        # Update the state of newly infected nodes
        for node in new_infected:
            G.nodes[node]['state'] = 'infected'

        # Recover some infected nodes based on the recovery rate
        infected_nodes = [node for node in G.nodes if G.nodes[node]['state'] == 'infected']
        recover_nodes = np.random.choice(infected_nodes, int(len(infected_nodes) * recovery_rate), replace=False)
        for node in recover_nodes:
            G.nodes[node]['state'] = 'recovered'

        # Dynamic visualization of the infection spread in the graph
        plt.figure(figsize=(8, 8))
        node_colors = ['red' if G.nodes[node]['state'] == 'infected' else
                       'green' if G.nodes[node]['state'] == 'recovered' else
                       'blue' for node in G.nodes]
        nx.draw(G, pos, node_color=node_colors, with_labels=False, node_size=100)
        plt.title(f'Day {day+1}: Infection Spread')

        # Adding a legend manually for clarity
        plt.scatter([], [], c='red', label='Infected')
        plt.scatter([], [], c='blue', label='Susceptible')
        plt.scatter([], [], c='green', label='Recovered')
        plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc="upper right", fontsize='large')

        # Show the plot and clear for the next day
        plt.show()
        clear_output(wait=True)

    # Final graph visualization
    plt.figure(figsize=(8, 8))
    node_colors = ['red' if G.nodes[node]['state'] == 'infected' else
                   'green' if G.nodes[node]['state'] == 'recovered' else
                   'blue' for node in G.nodes]
    nx.draw(G, pos, node_color=node_colors, with_labels=False, node_size=100)
    plt.title(f'Final Infection Spread after {days} Days')

    # Add legend to the final plot
    plt.scatter([], [], c='red', label='Infected')
    plt.scatter([], [], c='blue', label='Susceptible')
    plt.scatter([], [], c='green', label='Recovered')
    plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc="upper right", fontsize='large')

    plt.show()

    # Explanation of the simulation results
    print(f"Simulation complete after {days} days.")
    print("Explanation:")
    print("1. Red nodes: represent infected individuals who can spread the disease to their neighbors.")
    print("2. Blue nodes: represent susceptible individuals who have not yet been infected.")
    print("3. Green nodes: represent recovered individuals who can no longer spread or catch the disease.")
    print("\nThe graph simulates how an infection spreads through a population over time, with isolated and unisolated individuals.")
    print("Isolated individuals spread the disease less due to reduced contact, while vaccinated individuals have a reduced chance of infection.")

# Interactive sliders for the simulation parameters
interact(simulate_infection_graph,
         population_size=IntSlider(min=50, max=500, step=10, value=100, description='Population Size'),
         initial_infected=IntSlider(min=1, max=50, step=1, value=5, description='Initial Infected'),
         R0=FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='R0 (Infection Rate)'),
         recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Recovery Rate'),
         isolation_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description='Isolation Rate'),
         vaccination_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7, description='Vaccination Rate'),
         vaccine_efficacy=FloatSlider(min=0.5, max=1.0, step=0.01, value=0.9, description='Vaccine Efficacy'),
         days=IntSlider(min=1, max=30, step=1, value=15, description='Days'));


interactive(children=(IntSlider(value=100, description='Population Size', max=500, min=50, step=10), IntSlider…